In [1]:
%load_ext autoreload
%autoreload 2

import json, random, os
from tqdm import tqdm
import seaborn as sn 
import matplotlib.pyplot as plt
from bertviz import model_view
from dataclasses import dataclass, field
from tqdm import tqdm
from typing import Optional
import transformers
import torch
import json
from torch.utils.data import DataLoader
import os, json, random, pickle,re
import numpy as np
from huggingface_hub import login
from load_data.preprocess import GSMData, AquaData, StrategyQAData, StrategyQAData_Ours, CommonsenseQAData_Ours, TruthfulQAData_Ours
from model.generation_utils import make_sparse_mask
from model.load_model import MyAutoModelForCausalLM
from model.peft_model import MyPeftModelForCausalLM

/common/home/mj939/miniconda3/envs/reasonrag/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## load checkpoint

In [ ]:
checkpoint_path = "...../checkpoints/meta-llama/Llama-3.1-8B-Instruct/commonsenseqa_agent/step_type=memory-3-4-efficient=lora+prompt-tuning-lr=2e-4-soft-prompt=True/checkpoint-1466"
tokenizer = transformers.AutoTokenizer.from_pretrained(checkpoint_path) 
prompt_text = json.load(open(".../checkpoints/meta-llama/Llama-3.1-8B-Instruct/commonsenseqa_agent/step_type=memory-3-4-efficient=lora+prompt-tuning-lr=2e-4-soft-prompt=True/prompt_text.json"))
print(prompt_text)
special_tokens_list = []
for k in prompt_text:
    tokens = prompt_text[k].split('>')
    special_tokens_list += [tok+'>' for tok in tokens[:-1]]
prompt_tokens = tokenizer.convert_tokens_to_ids(special_tokens_list)
num_new_tokens = len(special_tokens_list)

model = MyAutoModelForCausalLM.from_pretrained(n_tokens=num_new_tokens,
            input_embedding_file=checkpoint_path + '/input_embeddings.pt',
            output_embedding_file=checkpoint_path + '/output_embeddings.pt',
            prompt_tokens=prompt_tokens,
            pretrained_model_name_or_path=checkpoint_path, #'meta-llama/Llama-2-7b-hf',
            parameter_efficient_mode='lora+prompt-tuning',
            cache_dir=None, torch_dtype=torch.float32, 
            device_map="cuda:0", load_in_8bit=True,
            offload_folder="offload", offload_state_dict = True)

In [ ]:
text = """Question: What room would you find many bookcases and is used for contemplation? Options: A.study B.house C.homw D.kitchen E.den 
<prefix_0><prefix_1><prefix_2><reason_0><reason_1><reason_2><reason_3>The question asks about a room where you would find many bookcases and which is used for contemplation. We need to identify the key features that describe this room.
<prefix_0><prefix_1><prefix_2><reason_0><reason_1><reason_2><reason_3>The key features are \"many bookcases\" and \"used for contemplation.\" These features are important in determining the type of room being described.
<prefix_0><prefix_1><prefix_2><memory_0><memory_1><memory_2><memory_3>A den is typically a small, intimate room used for reading, relaxation, or contemplation. It often contains many books and is designed as a quiet space. 
<prefix_0><prefix_1><prefix_2><reason_0><reason_1><reason_2><reason_3>Based on the key features and the knowledge that a den is used for contemplation and often contains many bookcases, the most suitable option is \"E. den.\" 
<prefix_0><prefix_1><prefix_2><answer_0><answer_1><answer_2><answer_3>The answer is: E
"""

inputs = tokenizer(text, padding=True, return_tensors='pt').to('cuda:0')
outputs = model(**inputs,  output_attentions=True)
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
data = outputs.attentions[-1][0][0].cpu().detach().numpy()

In [ ]:
ids = list(range(len(tokens)))
cots = []
start = 0
end = 0
for i in ids:
    if tokens[i] == "<prefix_0>":
        if start > 0:
            cots.append((start, i))
        start = i
cots.append((start, i+1))
print(cots)

In [ ]:
xticklabels = [token.replace("Ġ", "-") for token in tokens[cots[0][0]:cots[2][1]]]
yticklabels = [token.replace("Ġ", "-") for token in tokens[cots[0][0]:cots[2][1]]]

In [ ]:
# Changing the color palette and adding bold labels for both x and y axes

# Adjusting font scale for the heatmap and figure size
sn.set(font_scale=0.49)
plt.figure(figsize=(12, 10))

# Sample attention data for plotting
data = outputs.attentions[0][0][15].cpu().detach().numpy() # Example data for demonstration, replace with actual data when available

# Plot the heatmap with a different color palette and bold axis labels
hm = sn.heatmap(data=data[cots[0][0]:cots[2][1], cots[0][0]:cots[2][1]], 
                xticklabels=xticklabels, 
                yticklabels=yticklabels, 
                annot=False, )

  # Choosing YlGnBu for a visually appealing color palette


# Bold the tick labels on both x and y axes
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')
plt.savefig("heatmap_llama3-8b.pdf",bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
text = """Question: Was ship that recovered Apollo 13 named after a World War II battle?\n  
<prefix_0><prefix_1><prefix_2><memory_0><memory_1><memory_2><memory_3>The ship that recovered the Apollo 13 astronauts was the USS Iwo Jima. The USS Iwo Jima was named after the Battle of Iwo Jima, a pivotal battle in World War II
<prefix_0><prefix_1><prefix_2><reason_0><reason_1><reason_2><reason_3>Since the ship was named after the Battle of Iwo Jima, it is reasonable to conclude that the ship was named after a World War II battle.
<prefix_0><prefix_1><prefix_2><answer_0><answer_1><answer_2><answer_3>The answer is: True"""


inputs = tokenizer(text, padding=True, return_tensors='pt').to('cuda:0')
outputs = model(**inputs,  output_attentions=True)
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
data = outputs.attentions[-1][0][0].cpu().detach().numpy()

ids = list(range(len(tokens)))
cots = []
start = 0
end = 0
for i in ids:
    if tokens[i] == "<prefix_0>":
        if start > 0:
            cots.append((start, i))
        start = i
cots.append((start, i+1))
print(cots)
xticklabels = [token.replace("Ġ", "-") for token in tokens[cots[0][0]:cots[2][1]]]
yticklabels = [token.replace("Ġ", "-") for token in tokens[cots[0][0]:cots[2][1]]]

In [ ]:
# Changing the color palette and adding bold labels for both x and y axes

# Adjusting font scale for the heatmap and figure size
sn.set(font_scale=0.49)
plt.figure(figsize=(12, 10))

# Sample attention data for plotting
data = outputs.attentions[0][0][15].cpu().detach().numpy() # Example data for demonstration, replace with actual data when available

# Plot the heatmap with a different color palette and bold axis labels
hm = sn.heatmap(data=data[cots[0][0]:cots[2][1], cots[0][0]:cots[2][1]], 
                xticklabels=xticklabels, 
                yticklabels=yticklabels, 
                annot=False, )

  # Choosing YlGnBu for a visually appealing color palette


# Bold the tick labels on both x and y axes
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')
plt.savefig("heatmap_llama3-8b1.pdf",bbox_inches='tight')
plt.show()
plt.close()